# HAR LSTM training 

In [1]:
# Imports
import numpy as np
import os
from utils.utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

## Prepare data

In [2]:
X_train, labels_train, list_ch_train = read_data(data_path="/home/arasdar/datasets/har-data/", split="train") # train
X_test, labels_test, list_ch_test = read_data(data_path="/home/arasdar/datasets/har-data/", split="test") # test

assert list_ch_train == list_ch_test, "Mistmatch in channels!"

/home/arasdar/github/deep-learning-HAR/utils/utilities.py:37: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  X[:,:,i_ch] = dat_.as_matrix()


In [3]:
# # Standardize
# X_train, X_test = standardize(X_train, X_test) 

Train/Validation Split

In [4]:
X_tr, X_vld, lab_tr, lab_vld = train_test_split(X_train, labels_train, 
                                                stratify = labels_train,
                                                random_state = 123)

One-hot encoding:

In [6]:
y_tr = one_hot(lab_tr)
y_vld = one_hot(lab_vld)
y_test = one_hot(labels_test)

### Hyperparameters

In [7]:
# Imports
import tensorflow as tf

lstm_size = 27         # 3 times the amount of channels
lstm_layers = 2        # Number of layers
batch_size = 600       # Batch size
seq_len = 128          # Number of steps
learning_rate = 0.0001  # Learning rate (default is 0.001)
epochs = 1000

# Fixed
n_classes = 6
n_channels = 9

### Construct the graph
Placeholders

In [8]:
graph = tf.Graph()

# Construct placeholders
with graph.as_default():
    inputs_ = tf.placeholder(tf.float32, [None, seq_len, n_channels], name = 'inputs')
    labels_ = tf.placeholder(tf.float32, [None, n_classes], name = 'labels')
    keep_prob_ = tf.placeholder(tf.float32, name = 'keep')
    learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')

Construct inputs to LSTM

In [9]:
with graph.as_default():
    # Construct the LSTM inputs and LSTM cells
    lstm_in = tf.transpose(inputs_, [1,0,2]) # reshape into (seq_len, N, channels)
    lstm_in = tf.reshape(lstm_in, [-1, n_channels]) # Now (seq_len*N, n_channels)
    
    # To cells
    lstm_in = tf.layers.dense(lstm_in, lstm_size, activation=None) # or tf.nn.relu, tf.nn.sigmoid, tf.nn.tanh?
    
    # Open up the tensor into a list of seq_len pieces
    lstm_in = tf.split(lstm_in, seq_len, 0)
    
    # Add LSTM layers
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob_)
    cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)

Define forward pass, cost function and optimizer:

In [10]:
with graph.as_default():
    outputs, final_state = tf.contrib.rnn.static_rnn(cell, lstm_in, dtype=tf.float32,
                                                     initial_state = initial_state)
    
    # We only need the last output tensor to pass into a classifier
    logits = tf.layers.dense(outputs[-1], n_classes, name='logits')
    
    # Cost function and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels_))
    optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost) # No grad clipping
    
    #     # Grad clipping
    #     train_op = tf.train.AdamOptimizer(learning_rate_)

    #     gradients = train_op.compute_gradients(cost)
    #     capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
    #     optimizer = train_op.apply_gradients(capped_gradients)
    
    # Accuracy
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

### Train the network

In [11]:
if (os.path.exists('checkpoints') == False):
    !mkdir checkpoints

In [ ]:
validation_acc = []
validation_loss = []

train_acc = []
train_loss = []

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    
    for e in range(epochs):
        # Initialize 
        state = sess.run(initial_state)
        
        # Loop over batches
        for x,y in get_batches(X_tr, y_tr, batch_size):
            
            # Feed dictionary
            feed = {inputs_ : x, labels_ : y, keep_prob_ : 0.5, 
                    initial_state : state, learning_rate_ : learning_rate}
            
            loss, _ , state, acc = sess.run([cost, optimizer, final_state, accuracy], 
                                             feed_dict = feed)
            train_acc.append(acc)
            train_loss.append(loss)
            
            # Print at each 5 iters
            if (iteration % 5 == 0):
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Train loss: {:6f}".format(loss),
                      "Train acc: {:.6f}".format(acc))
            
            # Compute validation loss at every 25 iterations
            if (iteration%25 == 0):
                
                # Initiate for validation set
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                
                val_acc_ = []
                val_loss_ = []
                for x_v, y_v in get_batches(X_vld, y_vld, batch_size):
                    # Feed
                    feed = {inputs_ : x_v, labels_ : y_v, keep_prob_ : 1.0, initial_state : val_state}
                    
                    # Loss
                    loss_v, state_v, acc_v = sess.run([cost, final_state, accuracy], feed_dict = feed)
                    
                    val_acc_.append(acc_v)
                    val_loss_.append(loss_v)
                
                # Print info
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Validation loss: {:6f}".format(np.mean(val_loss_)),
                      "Validation acc: {:.6f}".format(np.mean(val_acc_)))
                
                # Store
                validation_acc.append(np.mean(val_acc_))
                validation_loss.append(np.mean(val_loss_))
            
            # Iterate 
            iteration += 1
    
    saver.save(sess,"checkpoints/har-lstm.ckpt")

Epoch: 0/1000 Iteration: 5 Train loss: 1.794995 Train acc: 0.201667
Epoch: 1/1000 Iteration: 10 Train loss: 1.785833 Train acc: 0.213333
Epoch: 1/1000 Iteration: 15 Train loss: 1.790771 Train acc: 0.198333
Epoch: 2/1000 Iteration: 20 Train loss: 1.782641 Train acc: 0.178333
Epoch: 2/1000 Iteration: 25 Train loss: 1.765518 Train acc: 0.223333
Epoch: 2/1000 Iteration: 25 Validation loss: 1.766853 Validation acc: 0.298333
Epoch: 3/1000 Iteration: 30 Train loss: 1.764966 Train acc: 0.230000
Epoch: 3/1000 Iteration: 35 Train loss: 1.769282 Train acc: 0.191667
Epoch: 4/1000 Iteration: 40 Train loss: 1.760452 Train acc: 0.250000
Epoch: 4/1000 Iteration: 45 Train loss: 1.747306 Train acc: 0.221667
Epoch: 5/1000 Iteration: 50 Train loss: 1.747209 Train acc: 0.236667
Epoch: 5/1000 Iteration: 50 Validation loss: 1.741095 Validation acc: 0.300000
Epoch: 6/1000 Iteration: 55 Train loss: 1.745167 Train acc: 0.251667
Epoch: 6/1000 Iteration: 60 Train loss: 1.740921 Train acc: 0.236667
Epoch: 7/1000 I

Epoch: 53/1000 Iteration: 480 Train loss: 1.021764 Train acc: 0.643333
Epoch: 53/1000 Iteration: 485 Train loss: 0.980003 Train acc: 0.651667
Epoch: 54/1000 Iteration: 490 Train loss: 0.993402 Train acc: 0.618333
Epoch: 54/1000 Iteration: 495 Train loss: 0.980677 Train acc: 0.645000
Epoch: 55/1000 Iteration: 500 Train loss: 0.964773 Train acc: 0.676667
Epoch: 55/1000 Iteration: 500 Validation loss: 0.865003 Validation acc: 0.731667
Epoch: 56/1000 Iteration: 505 Train loss: 0.993118 Train acc: 0.623333
Epoch: 56/1000 Iteration: 510 Train loss: 0.955679 Train acc: 0.648333
Epoch: 57/1000 Iteration: 515 Train loss: 0.965129 Train acc: 0.625000
Epoch: 57/1000 Iteration: 520 Train loss: 0.941284 Train acc: 0.683333
Epoch: 58/1000 Iteration: 525 Train loss: 0.970963 Train acc: 0.655000
Epoch: 58/1000 Iteration: 525 Validation loss: 0.837038 Validation acc: 0.747222
Epoch: 58/1000 Iteration: 530 Train loss: 0.953774 Train acc: 0.658333
Epoch: 59/1000 Iteration: 535 Train loss: 0.951342 Train 

Epoch: 105/1000 Iteration: 950 Validation loss: 0.562644 Validation acc: 0.805556
Epoch: 106/1000 Iteration: 955 Train loss: 0.662764 Train acc: 0.760000
Epoch: 106/1000 Iteration: 960 Train loss: 0.690417 Train acc: 0.753333
Epoch: 107/1000 Iteration: 965 Train loss: 0.670889 Train acc: 0.770000
Epoch: 107/1000 Iteration: 970 Train loss: 0.663883 Train acc: 0.783333
Epoch: 108/1000 Iteration: 975 Train loss: 0.701723 Train acc: 0.751667
Epoch: 108/1000 Iteration: 975 Validation loss: 0.555859 Validation acc: 0.802222
Epoch: 108/1000 Iteration: 980 Train loss: 0.625795 Train acc: 0.783333
Epoch: 109/1000 Iteration: 985 Train loss: 0.692706 Train acc: 0.765000
Epoch: 109/1000 Iteration: 990 Train loss: 0.709352 Train acc: 0.763333
Epoch: 110/1000 Iteration: 995 Train loss: 0.650171 Train acc: 0.765000
Epoch: 111/1000 Iteration: 1000 Train loss: 0.672656 Train acc: 0.751667
Epoch: 111/1000 Iteration: 1000 Validation loss: 0.540593 Validation acc: 0.809444
Epoch: 111/1000 Iteration: 1005 

Epoch: 156/1000 Iteration: 1410 Train loss: 0.529428 Train acc: 0.795000
Epoch: 157/1000 Iteration: 1415 Train loss: 0.587699 Train acc: 0.781667
Epoch: 157/1000 Iteration: 1420 Train loss: 0.552996 Train acc: 0.798333
Epoch: 158/1000 Iteration: 1425 Train loss: 0.602559 Train acc: 0.768333
Epoch: 158/1000 Iteration: 1425 Validation loss: 0.474936 Validation acc: 0.817222
Epoch: 158/1000 Iteration: 1430 Train loss: 0.546039 Train acc: 0.811667
Epoch: 159/1000 Iteration: 1435 Train loss: 0.601051 Train acc: 0.775000
Epoch: 159/1000 Iteration: 1440 Train loss: 0.575929 Train acc: 0.805000
Epoch: 160/1000 Iteration: 1445 Train loss: 0.549095 Train acc: 0.798333
Epoch: 161/1000 Iteration: 1450 Train loss: 0.546552 Train acc: 0.803333
Epoch: 161/1000 Iteration: 1450 Validation loss: 0.453055 Validation acc: 0.825556
Epoch: 161/1000 Iteration: 1455 Train loss: 0.548813 Train acc: 0.791667
Epoch: 162/1000 Iteration: 1460 Train loss: 0.564303 Train acc: 0.798333
Epoch: 162/1000 Iteration: 1465

Epoch: 207/1000 Iteration: 1870 Train loss: 0.466097 Train acc: 0.835000
Epoch: 208/1000 Iteration: 1875 Train loss: 0.513597 Train acc: 0.811667
Epoch: 208/1000 Iteration: 1875 Validation loss: 0.396288 Validation acc: 0.879444
Epoch: 208/1000 Iteration: 1880 Train loss: 0.475486 Train acc: 0.835000
Epoch: 209/1000 Iteration: 1885 Train loss: 0.511461 Train acc: 0.816667
Epoch: 209/1000 Iteration: 1890 Train loss: 0.509956 Train acc: 0.820000
Epoch: 210/1000 Iteration: 1895 Train loss: 0.482737 Train acc: 0.811667
Epoch: 211/1000 Iteration: 1900 Train loss: 0.451807 Train acc: 0.855000
Epoch: 211/1000 Iteration: 1900 Validation loss: 0.392358 Validation acc: 0.882778
Epoch: 211/1000 Iteration: 1905 Train loss: 0.492769 Train acc: 0.825000
Epoch: 212/1000 Iteration: 1910 Train loss: 0.469521 Train acc: 0.841667
Epoch: 212/1000 Iteration: 1915 Train loss: 0.477185 Train acc: 0.825000
Epoch: 213/1000 Iteration: 1920 Train loss: 0.529963 Train acc: 0.808333
Epoch: 213/1000 Iteration: 1925

...

Epoch: 666/1000 Iteration: 5995 Train loss: 0.141979 Train acc: 0.958333
Epoch: 666/1000 Iteration: 6000 Train loss: 0.136453 Train acc: 0.958333
Epoch: 666/1000 Iteration: 6000 Validation loss: 0.124754 Validation acc: 0.956111
Epoch: 667/1000 Iteration: 6005 Train loss: 0.138128 Train acc: 0.953333
Epoch: 667/1000 Iteration: 6010 Train loss: 0.140928 Train acc: 0.958333
Epoch: 668/1000 Iteration: 6015 Train loss: 0.170555 Train acc: 0.936667
Epoch: 668/1000 Iteration: 6020 Train loss: 0.142415 Train acc: 0.958333
Epoch: 669/1000 Iteration: 6025 Train loss: 0.183371 Train acc: 0.933333
Epoch: 669/1000 Iteration: 6025 Validation loss: 0.127952 Validation acc: 0.955000
Epoch: 669/1000 Iteration: 6030 Train loss: 0.160821 Train acc: 0.948333
Epoch: 670/1000 Iteration: 6035 Train loss: 0.156983 Train acc: 0.945000
Epoch: 671/1000 Iteration: 6040 Train loss: 0.134420 Train acc: 0.955000
Epoch: 671/1000 Iteration: 6045 Train loss: 0.145733 Train acc: 0.951667
Epoch: 672/1000 Iteration: 6050

Epoch: 716/1000 Iteration: 6450 Validation loss: 0.127734 Validation acc: 0.955000
Epoch: 717/1000 Iteration: 6455 Train loss: 0.131717 Train acc: 0.958333
Epoch: 717/1000 Iteration: 6460 Train loss: 0.127185 Train acc: 0.953333
Epoch: 718/1000 Iteration: 6465 Train loss: 0.159265 Train acc: 0.948333
Epoch: 718/1000 Iteration: 6470 Train loss: 0.140264 Train acc: 0.956667
Epoch: 719/1000 Iteration: 6475 Train loss: 0.195431 Train acc: 0.925000
Epoch: 719/1000 Iteration: 6475 Validation loss: 0.129831 Validation acc: 0.953333
Epoch: 719/1000 Iteration: 6480 Train loss: 0.166245 Train acc: 0.940000
Epoch: 720/1000 Iteration: 6485 Train loss: 0.154873 Train acc: 0.943333
Epoch: 721/1000 Iteration: 6490 Train loss: 0.124066 Train acc: 0.955000
Epoch: 721/1000 Iteration: 6495 Train loss: 0.146602 Train acc: 0.956667
Epoch: 722/1000 Iteration: 6500 Train loss: 0.132529 Train acc: 0.956667
Epoch: 722/1000 Iteration: 6500 Validation loss: 0.126171 Validation acc: 0.955555
Epoch: 722/1000 Itera

Epoch: 767/1000 Iteration: 6910 Train loss: 0.159671 Train acc: 0.950000
Epoch: 768/1000 Iteration: 6915 Train loss: 0.159479 Train acc: 0.936667
Epoch: 768/1000 Iteration: 6920 Train loss: 0.122884 Train acc: 0.958333
Epoch: 769/1000 Iteration: 6925 Train loss: 0.159173 Train acc: 0.936667
Epoch: 769/1000 Iteration: 6925 Validation loss: 0.126964 Validation acc: 0.954444
Epoch: 769/1000 Iteration: 6930 Train loss: 0.149105 Train acc: 0.960000
Epoch: 770/1000 Iteration: 6935 Train loss: 0.160918 Train acc: 0.936667
Epoch: 771/1000 Iteration: 6940 Train loss: 0.126021 Train acc: 0.956667
Epoch: 771/1000 Iteration: 6945 Train loss: 0.139311 Train acc: 0.935000
Epoch: 772/1000 Iteration: 6950 Train loss: 0.132435 Train acc: 0.960000
Epoch: 772/1000 Iteration: 6950 Validation loss: 0.124123 Validation acc: 0.952778
Epoch: 772/1000 Iteration: 6955 Train loss: 0.139378 Train acc: 0.951667
Epoch: 773/1000 Iteration: 6960 Train loss: 0.158773 Train acc: 0.946667
Epoch: 773/1000 Iteration: 6965

Epoch: 818/1000 Iteration: 7370 Train loss: 0.125138 Train acc: 0.965000
Epoch: 819/1000 Iteration: 7375 Train loss: 0.158065 Train acc: 0.930000
Epoch: 819/1000 Iteration: 7375 Validation loss: 0.126700 Validation acc: 0.951667
Epoch: 819/1000 Iteration: 7380 Train loss: 0.147595 Train acc: 0.948333
Epoch: 820/1000 Iteration: 7385 Train loss: 0.132957 Train acc: 0.951667
Epoch: 821/1000 Iteration: 7390 Train loss: 0.133888 Train acc: 0.945000
Epoch: 821/1000 Iteration: 7395 Train loss: 0.124400 Train acc: 0.955000
Epoch: 822/1000 Iteration: 7400 Train loss: 0.135411 Train acc: 0.953333
Epoch: 822/1000 Iteration: 7400 Validation loss: 0.122138 Validation acc: 0.956111
Epoch: 822/1000 Iteration: 7405 Train loss: 0.120355 Train acc: 0.958333
Epoch: 823/1000 Iteration: 7410 Train loss: 0.155215 Train acc: 0.945000
Epoch: 823/1000 Iteration: 7415 Train loss: 0.120246 Train acc: 0.963333
Epoch: 824/1000 Iteration: 7420 Train loss: 0.166659 Train acc: 0.938333
Epoch: 824/1000 Iteration: 7425

Epoch: 869/1000 Iteration: 7825 Validation loss: 0.124687 Validation acc: 0.951667
Epoch: 869/1000 Iteration: 7830 Train loss: 0.143709 Train acc: 0.951667
Epoch: 870/1000 Iteration: 7835 Train loss: 0.130293 Train acc: 0.946667
Epoch: 871/1000 Iteration: 7840 Train loss: 0.139059 Train acc: 0.951667
Epoch: 871/1000 Iteration: 7845 Train loss: 0.134774 Train acc: 0.948333
Epoch: 872/1000 Iteration: 7850 Train loss: 0.140196 Train acc: 0.950000
Epoch: 872/1000 Iteration: 7850 Validation loss: 0.123512 Validation acc: 0.956111
Epoch: 872/1000 Iteration: 7855 Train loss: 0.129048 Train acc: 0.956667
Epoch: 873/1000 Iteration: 7860 Train loss: 0.150748 Train acc: 0.943333
Epoch: 873/1000 Iteration: 7865 Train loss: 0.115698 Train acc: 0.961667
Epoch: 874/1000 Iteration: 7870 Train loss: 0.155244 Train acc: 0.943333
Epoch: 874/1000 Iteration: 7875 Train loss: 0.131323 Train acc: 0.958333
Epoch: 874/1000 Iteration: 7875 Validation loss: 0.123792 Validation acc: 0.956111
Epoch: 875/1000 Itera

In [ ]:
# Plot training and test loss
t = np.arange(iteration-1)

plt.figure(figsize = (6,6))
plt.plot(t, np.array(train_loss), 'r-', t[t % 25 == 0], np.array(validation_loss), 'b*')
plt.xlabel("iteration")
plt.ylabel("Loss")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot Accuracies
plt.figure(figsize = (6,6))

plt.plot(t, np.array(train_acc), 'r-', t[t % 25 == 0], validation_acc, 'b*')
plt.xlabel("iteration")
plt.ylabel("Accuray")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

## Evaluate on test set

In [ ]:
test_acc = []

with tf.Session(graph=graph) as sess:
    # Restore
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    
    for x_t, y_t in get_batches(X_test, y_test, batch_size):
        feed = {inputs_: x_t,
                labels_: y_t,
                keep_prob_: 1,
                initial_state: test_state}
        
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.6f}".format(np.mean(test_acc)))